# The first cell's code will be in a separate script

In [ ]:
import pandas as pd 
from pathlib import Path
import ipywidgets as widgets
from autumn.projects.sm_covid2.common_school.output_plots import country_highlight as ch

from autumn.projects.sm_covid2.common_school.runner_tools import INCLUDED_COUNTRIES

def get_analysis_widgets():
    dropdown_countries = [(c, iso3) for iso3, c in INCLUDED_COUNTRIES['all'].items()]
    iso3_widget = widgets.Dropdown(
        options=dropdown_countries,
        value="AUS",
        description='Country:',
    )
    analysis_widget = widgets.Dropdown(
        options=[
            ("Base case", "main"), 
            ("SA1: Increased household transmission during closures", "increased_hh_contacts"),
            ("SA2: Without Google mobility data", "no_google_mobility")    
        ],
        value="main",
        description='Analysis:',
    )

    return iso3_widget, analysis_widget


def get_derived_output_widget():
    return widgets.Dropdown(
        options=['cumulative_incidence', 'cumulative_incidenceXstrain_delta',
        'cumulative_incidenceXstrain_omicron',
        'cumulative_incidenceXstrain_wild_type',
        'cumulative_incidence_propXstrain_delta',
        'cumulative_incidence_propXstrain_omicron',
        'cumulative_incidence_propXstrain_wild_type',
        'cumulative_infection_deaths', 'elderly_incidence_prop',
        'ever_infected', 'hospital_admissions', 'hospital_occupancy',
        'incidence', 'incidenceXagegroup_0', 'incidenceXagegroup_15',
        'incidenceXagegroup_25', 'incidenceXagegroup_50',
        'incidenceXagegroup_70', 'infection_deaths', 'infection_deaths_ma7',
        'n_immune_unvaccinated', 'n_immune_vaccinated',
        'peak_hospital_occupancy', 'prop_ever_infected',
        'prop_ever_infected_age_matched', 'prop_immune_unvaccinated',
        'prop_immune_vaccinated',
        'total_population', 'transformed_random_process'
        ],
        value="incidence",
        description='Output:',
    )



def load_analysis_outputs(iso3, analysis):    
    folder_path = Path.cwd() / analysis / iso3

    uncertainty_dfs = {sc: pd.read_parquet(folder_path/ f"uncertainty_df_{sc}.parquet") for sc in ["baseline", "scenario_1"]}
    diff_quantiles_df = pd.read_parquet(folder_path / f"diff_quantiles_df.parquet")
    derived_outputs = pd.read_pickle(folder_path / "derived_outputs.pickle") 

    return uncertainty_dfs, diff_quantiles_df, derived_outputs


def get_diff_quantiles_table(diff_quantiles_df):
    f_diff_quantiles_df = - diff_quantiles_df[
        ['cases_averted', 'cases_averted_relative', 'delta_hospital_peak_relative', 'deaths_averted', 'deaths_averted_relative']
    ] # use "-" so positive nbs indicate positive effect of closures

    for col in f_diff_quantiles_df.columns:
        if col.endswith("_relative"):
            f_diff_quantiles_df[col] = round(100. * f_diff_quantiles_df[col], 1)
        else:
            f_diff_quantiles_df[col] = (round(f_diff_quantiles_df[col])).astype("int")

    # rename columns
    rename_map = {
        "cases_averted": "N infections averted",
        "cases_averted_relative": "% infections averted",
        "deaths_averted": "N deaths averted",
        "deaths_averted_relative": "% deaths averted",
        # "delta_hospital_peak": "Hospital peak reduction",
        "delta_hospital_peak_relative": "% hospital peak reduction"
    }
    f_diff_quantiles_df.rename(mapper=rename_map, inplace=True, axis=1)

    # rename and reorder quantile index
    f_diff_quantiles_df.rename(index=lambda q: f"{round(100. * (1. - q), 1)}%" ,inplace=True)
    f_diff_quantiles_df.index.name = "percentile"
    f_diff_quantiles_df = f_diff_quantiles_df.iloc[::-1]

    return f_diff_quantiles_df


def plot_derived_outputs(derived_outputs, output):
    sc_names = {"baseline": "Historical", "scenario_1": "Counterfactual (schools open)"}
    df = pd.DataFrame({sc_names[sc]: derived_outputs[sc][output] for sc in derived_outputs})
    ax = df.plot(labels={"value": output,"variable": "Scenario"})
    ax.show()

# The actual notebook content will start here

## Imports

In [ ]:
from IPython.display import display
from autumn.projects.sm_covid2.common_school.output_plots import country_highlight as ch
import pandas as pd
pd.options.plotting.backend = "plotly"

## Select country and analysis

In [ ]:
iso3_widget, analysis_widget = get_analysis_widgets()
display(iso3_widget, analysis_widget)

## Redo country highlight figure, as provided in the manuscript

In [ ]:
uncertainty_dfs, diff_quantiles_df, derived_outputs = load_analysis_outputs(iso3_widget.value, analysis_widget.value)
fig = ch.make_country_highlight_figure(iso3_widget.value, uncertainty_dfs, diff_quantiles_df, derived_outputs, include_country_name=True)

## Relative outcomes

* Positive values indicate a positive effect of school closures on the relevant indicator.
* Negative values indicate that school closures exacerbated the relevant COVID-19 indicator.

In [ ]:
df = get_diff_quantiles_table(diff_quantiles_df)
df

## Model output timeseries 
Outputs obtained with maximum a-posteriori parameter set

In [ ]:
output_widget = get_derived_output_widget()
display(output_widget)

In [ ]:
plot_derived_outputs(derived_outputs, output_widget.value)